In [1414]:
import statistics
import queue
from collections import namedtuple
import numpy as np
import random
from tqdm import tqdm
import networkx as nx
from copy import deepcopy

from collections import Counter


In [1423]:
with open('./data/gc_50_3', 'r') as input_data_file:
    input_data = input_data_file.read()
    
lines = input_data.split('\n')

first_line = lines[0].split()
node_count = int(first_line[0])
edge_count = int(first_line[1])

edges = []
for i in range(1, edge_count + 1):
    line = lines[i]
    parts = line.split()
    edges.append((int(parts[0]), int(parts[1])))

In [1424]:
node_count, edge_count

(50, 350)

In [1425]:
def reset_coloring(max_colors):

    temp = dict()
    
    for idx in range(node_count):
        temp[idx] = list(range(max_colors))

    return temp

In [1426]:
mapping = dict()
    
for idx in range(node_count):
    mapping[idx] = []

for i, j in edges:
    mapping[i].append(j)
    mapping[j].append(i)

coloring = reset_coloring(node_count)

In [1434]:
# get 17
# sorted(mapping.items(), key=lambda x: len(x[1]), reverse=True)[0]
clique_set = set()

for find_clique_for in tqdm(mapping):
    for elements in mapping[find_clique_for]:
        found = set(mapping[elements]).intersection(set(mapping[find_clique_for]))
        for ele in found:
            clique_set.add(frozenset({ele, elements, find_clique_for}))
            # print(find_clique_for, elements, found)


# unique_cliques = [list(t) for t in set(tuple(lst) for lst in clique_set)]
# unique_cliques = sorted(unique_cliques, key=lambda x: len(x), reverse=True)

100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 12497.93it/s]


In [1435]:
connections = dict(zip(list(range(node_count)), [0]*node_count))
for elements in clique_set:
    for ele in elements:
        connections[ele] += 1
        
# connections = list(dict(sorted(connections.items(), key=lambda x: x[1], reverse=True)).keys())
connections = dict(sorted(connections.items(), key=lambda x: x[1], reverse=True))

In [1436]:
# scan_list = [45]
# for k, v in dict(sorted(connections.items(), key=lambda x: x[1], reverse=True)).items():
#     if k in mapping[45]:
#         scan_list.append(k)

# for k, v in dict(sorted(connections.items(), key=lambda x: x[1], reverse=True)).items():
#     if k not in scan_list:
#         scan_list.append(k)

In [1437]:
def is_valid_coloring(graph, coloring, node, color):
    graph = deepcopy(graph)
    for neighbor in graph[node]:
        if isinstance(coloring[neighbor], list):
            continue
        if coloring[neighbor] == color:
            return False
    del graph   
    return True
    

def calculate_conflicts(graph, coloring):
    conflicts = 0
    graph = deepcopy(graph)
    for node in graph:
        for neighbor in graph[node]:
            if isinstance(coloring[neighbor], list):
                continue
            if coloring[node] == coloring[neighbor]:
                conflicts += 1
            if node in  graph[neighbor]:
                graph[neighbor].remove(node)
    del graph
    return conflicts


def print_conflicts(graph, coloring):
    conflicts = 0
    graph = deepcopy(graph)
    for node in graph:
        for neighbor in graph[node]:
            if isinstance(coloring[neighbor], list):
                continue
            if coloring[node] == coloring[neighbor]:
                print(node, neighbor)
                conflicts += 1
            if node in  graph[neighbor]:
                graph[neighbor].remove(node)
    del graph
    pass


def get_conflict_nodes(graph, coloring):
    conflict_nodes = []
    graph = deepcopy(graph)
    for node in graph:
        for neighbor in graph[node]:
            if isinstance(coloring[neighbor], list):
                continue
            if coloring[node] == coloring[neighbor]:
                conflict_nodes.append(node)
                conflict_nodes.append(neighbor)
            if node in  graph[neighbor]:
                graph[neighbor].remove(node)
    del graph
    return list(set(conflict_nodes))


def least_conflict_swap(graph, coloring, conflicts):
    graph = deepcopy(graph)
    coloring = deepcopy(coloring)

    least_conflict = len(conflicts)
    
    for conflict in conflicts:
        for swap_node in graph[conflict]:
            swap_color1 = conflict
            swap_color2 = swap_node
            temp = coloring[swap_color1]
            coloring[swap_color1]= coloring[swap_color2]
            coloring[swap_color2] = temp

            current_conflicts = calculate_conflicts(graph, coloring)

            if current_conflicts <= least_conflict:
                return swap_color1, swap_color2
            else:
                swap_color1 = swap_node
                swap_color2 = conflict
                temp = coloring[swap_color1]
                coloring[swap_color1]= coloring[swap_color2]
                coloring[swap_color2] = temp
    del graph
    del coloring
    pass

In [1450]:
optimal_conflicts = edge_count ## Expecting 0
optimal_coloring = None
optimal_colors = node_count ## Expecting least number

In [1451]:
coloring = reset_coloring(node_count)
explored_colors = {}

In [1452]:
max_colors = 6

for set_node in connections:
    
    for set_color in range(max_colors):
        if is_valid_coloring(mapping, coloring, set_node, set_color):
            break
        else:
            set_color = random.randint(0, max_colors-1)
    
    coloring[set_node] = set_color
    
current_conflicts = calculate_conflicts(mapping, coloring)

if current_conflicts < optimal_conflicts:
    optimal_coloring = deepcopy(coloring)
    optimal_conflicts = current_conflicts 

# set_color
optimal_conflicts, len(set(optimal_coloring.values()))

(8, 6)

In [1441]:
optimal_conflicts, len(set(optimal_coloring.values()))

(8, 6)

In [1301]:
optimal_coloring

{0: 0,
 1: 5,
 2: 1,
 3: 2,
 4: 2,
 5: 4,
 6: 0,
 7: 5,
 8: 4,
 9: 5,
 10: 2,
 11: 4,
 12: 0,
 13: 0,
 14: 4,
 15: 4,
 16: 4,
 17: 3,
 18: 0,
 19: 4,
 20: 1,
 21: 0,
 22: 4,
 23: 1,
 24: 3,
 25: 1,
 26: 3,
 27: 0,
 28: 5,
 29: 2,
 30: 1,
 31: 3,
 32: 0,
 33: 5,
 34: 2,
 35: 3,
 36: 0,
 37: 3,
 38: 0,
 39: 0,
 40: 0,
 41: 1,
 42: 5,
 43: 0,
 44: 3,
 45: 1,
 46: 2,
 47: 3,
 48: 2,
 49: 1}

In [1312]:
# all_conflict_nodes = get_conflict_nodes(mapping, optimal_coloring)
# least_conflict_swap(mapping, optimal_coloring, all_conflict_nodes)

In [1335]:
calculate_conflicts(mapping, optimal_coloring)

13

In [1314]:
mapping[43]

[0, 4, 5, 6, 8, 9, 12, 19, 23, 28, 29, 33, 34, 37, 44, 47]

In [1194]:
for make_change in get_conflict_nodes(mapping, optimal_coloring):
    
    temp_color = optimal_coloring[make_change]
    base_conflict = calculate_conflicts(mapping, optimal_coloring)
    
    for color in range(6):
        optimal_coloring[make_change] = color
        current_conflict = calculate_conflicts(mapping, optimal_coloring)
        if current_conflict < base_conflict:
            base_conflict = current_conflict
            break
        optimal_coloring[make_change] = temp_color

In [1195]:
calculate_conflicts(mapping, optimal_coloring)

2

In [1196]:
for node in get_conflict_nodes(mapping, optimal_coloring):
    print(node, optimal_coloring[node])

27 5
44 3
47 5
15 3


In [1198]:
optimal_coloring[15]

3

In [1185]:
mapping[15]

[1, 2, 4, 6, 9, 11, 13, 16, 19, 27, 30, 31, 33, 44, 48, 49]

In [1197]:
optimal_coloring[1]

2

In [754]:
# for explore_node in get_conflict_nodes(mapping, optimal_coloring):
#     for color in range(6):
#         if is_valid_coloring(mapping, optimal_coloring, explore_node, color):
#             print(explore_node, color, is_valid_coloring(mapping, optimal_coloring, explore_node, color))
#             optimal_coloring[explore_node] = color

In [1409]:
# base_node = 25
iterations = 10
escape = False

with tqdm(total=iterations, desc="Iterating") as progress_bar:
    while iterations > 0:
        shuffle_list = get_conflict_nodes(mapping, optimal_coloring)
        random.shuffle(shuffle_list)
        for base_node in shuffle_list:
            for swap_node in mapping[base_node]:
                optimal_conflicts = calculate_conflicts(mapping, optimal_coloring)
                
                temp = optimal_coloring[base_node]
                optimal_coloring[base_node] = optimal_coloring[swap_node]
                optimal_coloring[swap_node] = temp
                
                if calculate_conflicts(mapping, optimal_coloring) >= optimal_conflicts:
                    temp = optimal_coloring[swap_node]
                    optimal_coloring[swap_node] = optimal_coloring[base_node]
                    optimal_coloring[base_node] = temp
    
                if calculate_conflicts(mapping, optimal_coloring) == 0:
                    escape = True
                    break
            
            if escape:
                break
                
        if escape:
            break
            
        iterations -= 1
        progress_bar.update(1)
    
    # print(calculate_conflicts(mapping, optimal_coloring))

Iterating: 100%|███████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.10it/s]


In [1410]:
print_conflicts(mapping, optimal_coloring)

6 28
7 9
14 29
15 31
15 49
20 40
23 42
29 43


In [1411]:
calculate_conflicts(mapping, optimal_coloring)

8

In [1413]:
optimal_coloring[49]

3

In [371]:
max_colors = 17
for default_color in tqdm(range(max_colors)):
    coloring = reset_coloring(node_count)
    for set_node in connections:
        for set_color in range(max_colors):
            if is_valid_coloring(mapping, coloring, set_node, set_color):
                break
            # else:
            #     set_color = default_color
        
        coloring[set_node] = set_color
        
    current_conflicts = calculate_conflicts(mapping, coloring)
    
    if current_conflicts < optimal_conflicts:
        optimal_coloring = deepcopy(coloring)
        optimal_conflicts = current_conflicts 



# max_colors = 6

# for _ in tqdm(range(10)):
    
#     coloring = reset_coloring(node_count)
    
#     # random.shuffle(connections)
    
#     for set_node in connections:
        
#         for set_color in range(max_colors):
#             if is_valid_coloring(mapping, coloring, set_node, set_color):
#                 break
#             else:
#                 set_color = random.randint(0, max_colors-1)
        
#         coloring[set_node] = set_color
        
#     current_conflicts = calculate_conflicts(mapping, coloring)
    
#     if current_conflicts < optimal_conflicts:
#         optimal_coloring = deepcopy(coloring)
#         optimal_conflicts = current_conflicts 

# set_color
optimal_conflicts, len(set(optimal_coloring.values()))

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:23<00:00,  1.31s/it]


(21, 18)

In [329]:
optimal_conflicts, len(set(optimal_coloring.values()))

(3, 6)

In [330]:
calculate_conflicts(mapping, optimal_coloring)

3

In [331]:
print_conflicts(mapping, optimal_coloring)

8 28
28 36
40 46


In [332]:
temp = optimal_coloring
mapping[8]

[0, 3, 11, 16, 17, 20, 27, 28, 31, 32, 34, 41, 43, 47]

In [ ]:
optimal_coloring[32] = 1

In [ ]:
for explore_node in get_conflict_nodes(mapping, optimal_coloring):
    for color in range(6):
        if is_valid_coloring(mapping, optimal_coloring, explore_node, color):
            print(explore_node, color, is_valid_coloring(mapping, optimal_coloring, explore_node, color))
            optimal_coloring[explore_node] = color

In [ ]:
calculate_conflicts(mapping, optimal_coloring)

In [ ]:

## Get most conflicts - move up the local minima

## check list
## dont use visited colors
## use the next best and see if you get less conflicts
## use that and adjust again
# explored_node = []
# while True:

## go up the tree with most conflicts. dive down and see least conflicts.
## if non found do it again
    
for explore_node in get_conflict_nodes(mapping, optimal_coloring):
    default_color = optimal_coloring[explore_node]
    
    for color in range(6):
        optimal_coloring[explore_node] = color
        # if calculate_conflicts(mapping, optimal_coloring) < 3:
        print(explore_node, color, calculate_conflicts(mapping, optimal_coloring))
        # if is_valid_coloring(mapping, optimal_coloring, explore_node, color):
        #     print(explore_node, color, is_valid_coloring(mapping, optimal_coloring, explore_node, color))
            # optimal_coloring[explore_node] = color

    optimal_coloring[explore_node] = default_color

In [ ]:
optimal_coloring[49] = 3
calculate_conflicts(mapping, optimal_coloring)

In [ ]:
36 4 3
36 5 3
8 2 3
8 4 3
40 1 3
40 4 3
40 5 3
47 2 3
49 3 3
49 5 3
20 1 3
20 3 3
20 4 3

In [1421]:
connections

{0: 0,
 1: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 18: 0,
 19: 0,
 2: 1,
 11: 1,
 17: 1}